# Querying Data from StreamCI

## 📡 Set URL, Request Headers, and Authentication Information

To retrieve data, we'll send a query to the `/query` endpoint. The response will be returned in [NDJSON](http://ndjson.org/) (Newline-Delimited JSON) format, so we’ll parse each line into a JSON object.

---

#### 🧠 `query(payload)` – Send a Query and Parse NDJSON Response

In [ ]:
import requests
import json

URL = "https://api.streamci.org/query"
HEADERS = {"Content-Type": "application/json"}
AUTH_INFO = {
    "target": "userXX",
    "authtype": "secret",
    "secret_key": ""
}

def query(payload):
    response = requests.post(URL, json=payload, headers=HEADERS)
    ndjson_text = response.text  # Get response as raw text (NDJSON)
    lines = ndjson_text.strip().splitlines()  # Split by line
    json_array = [json.loads(line) for line in lines] # Parse each line into a JSON object
    return json_array

---

## 📦 Sample Query Payload

To build the payload, define the `auth` section for authentication and a `request` section that specifies the query method and options. 

In the `request`, you can include filters (`query`), selected fields (`project`), sorting rules (`sort`), and record limit (`limit`).


- **`query`**: Specifies the filter criteria to select records. An empty `{}` returns all records.
- **`project`**: Defines which fields to include in the result. Useful for reducing payload size.
- **`sort`**: Determines the order of the results based on one or more fields (`1` for ascending, `-1` for descending).
- **`limit`**: Can be added to restrict the number of records returned.




In [ ]:
payload = {
    "auth": AUTH_INFO,
    "request": {
        "method": "query",
        "query": {},                                     # Match all records
    }
}
response = query(payload)
print("Responses: ", len(response), "records")

In [ ]:
payload = {
    "auth": AUTH_INFO,
    "request": {
        "method": "query",
        "query": {
            "time": {
                "$gte": {"$date": "2025-08-04T12:00:00.000Z"}, 
                "$lt": {"$date": "2025-08-04T13:00:00.000Z"}  # Filter with the "time" range
            }
        },
    }
}
response = query(payload)
print("Responses: ", len(response), "records")

In [ ]:
payload = {
    "auth": AUTH_INFO,
    "request": {
        "method": "query",
        "query": {
            "time": {
                "$gte": {"$date": "2025-08-04T12:00:00.000Z"}, 
                "$lt": {"$date": "2025-08-04T13:00:00.000Z"}  # Filter with the "time" range
            },
            "deviceID": "sensor1"                             # Filter with the deviceID == "sensor1"
        },
    }
}
response = query(payload)
print("Responses: ", len(response), "records")
print("Sample: ", response[0])

In [ ]:
payload = {
    "auth": AUTH_INFO,
    "request": {
        "method": "query",
        "query": {
            "time": {
                "$gte": {"$date": "2025-08-04T12:00:00.000Z"}, 
                "$lt": {"$date": "2025-08-04T13:00:00.000Z"}  # Filter with the "time" range
            },
            "deviceID": "sensor1"                             # Filter with the deviceID == "sensor1"
        },
        "project": ["time", "deviceID", "val1"],              # Select fields to be included: "time", "deviceID", and "val1"
    }
}
response = query(payload)
print("Responses: ", len(response), "records")
print("Sample: ", response[0])

In [ ]:
payload = {
    "auth": AUTH_INFO,
    "request": {
        "method": "query",
        "query": {
            "time": {
                "$gte": {"$date": "2025-08-04T12:00:00.000Z"}, 
                "$lt": {"$date": "2025-08-04T13:00:00.000Z"}  # Filter with the "time" range
            },
            "deviceID": "sensor1"                             # Filter with the deviceID == "sensor1"
        },
        "project": ["time", "deviceID", "val1"],              # Select fields to be included: "time", "deviceID", and "val1"
        "sort": {"time":-1},                                  # Sort by "time" in decending order
        "limit": 1                                            # limit the number of records returned
    }
}
response = query(payload)
print("Responses: ", len(response), "records")
print("Sample: ", response[0])

In [ ]:
payload = {
    "auth": AUTH_INFO,
    "request": {
        "method": "query",
        "query": {},                                     
        "project": ["time", "deviceID", "val1", "val2"],
    }
}
response = query(payload)
print("Responses: ", len(response), "records")

---

## 📊 Basic Data Analysis of Queried Results

After retrieving data from the `query` method, we can perform a simple analysis to understand sensor behavior.

#### 🧮 Convert to DataFrame and Compute Statistics

The code below:
- Converts the list of JSON objects into a Pandas DataFrame
- Parses the `time` column as datetime and convert the time zone
- Groups by `deviceID` and summarizes `val1` and `val2`


This provides insights like **mean**, **standard deviation**, **min/max**, and **quartiles** for each sensor’s values.

In [ ]:
import pandas as pd

# Convert to DataFrame
df = pd.DataFrame(response)

# Convert 'time' to datetime and localize to UTC
df["time"] = pd.to_datetime(df["time"], utc=True)

# Convert to US Eastern Time
df["time"] = df["time"].dt.tz_convert("US/Eastern")

# Group by deviceID and describe val1 and val2
summary = df.groupby("deviceID")[["val1", "val2"]].describe()

# Print summary
print(summary.to_string())

---

## 📈 Visualize Sensor Readings Over Time

We can plot `val2` over time to observe how each sensor behaves visually.

#### 🖼️ Scatter Plot by Sensor

The following code:

- Loops through each unique `deviceID`
- Filters the DataFrame for that device
- Uses `matplotlib.pyplot.scatter()` to plot `val2` against time
- Applies a small dot size (`s=3`) and semi-transparency (`alpha=0.5`) for clarity


In [ ]:
import matplotlib.pyplot as plt

# Plot dots for each sensor
plt.figure(figsize=(10, 5))
for sensor in df["deviceID"].unique():
    subset = df[df["deviceID"] == sensor]
    plt.scatter(subset["time"], subset["val2"], label=sensor, s=3, alpha=0.5)

plt.xlabel("Time (US/Eastern)")
plt.ylabel("val2")
plt.title("val2 Over Time by Sensor")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()